In [2]:
# -*- coding:utf-8 -*-
"""数据增强
   1. 翻转变换 flip
   2. 随机修剪 random crop
   3. 色彩抖动 color jittering
   4. 平移变换 shift
   5. 尺度变换 scale
   6. 对比度变换 contrast
   7. 噪声扰动 noise
   8. 旋转变换/反射变换 Rotation/reflection
"""

from PIL import Image, ImageEnhance, ImageOps, ImageFile
import numpy as np
import random
import threading, os, time
import logging

[//]: # (Image References)

[image1]: ./examples/visualization.jpg "Visualization"
[image2]: ./examples/grayscale.jpg "Grayscaling"
[image3]: ./examples/random_noise.jpg "Random Noise"
[image4]: ./examples/placeholder.png "Traffic Sign 1"
[image5]: ./examples/placeholder.png "Traffic Sign 2"
[image6]: ./examples/placeholder.png "Traffic Sign 3"
[image7]: ./examples/placeholder.png "Traffic Sign 4"
[image8]: ./examples/placeholder.png "Traffic Sign 5"

In [1]:

logger = logging.getLogger(__name__)
ImageFile.LOAD_TRUNCATED_IMAGES = True


class DataAugmentation:
    """
    包含数据增强的八种方式
    """


    def __init__(self):
        pass

    @staticmethod
    def openImage(image):
        return Image.open(image, mode="r")

    @staticmethod
    def randomRotation(image, mode=Image.BICUBIC):
        """
         对图像进行随机任意角度(0~360度)旋转
        :param mode 邻近插值,双线性插值,双三次B样条插值(default)
        :param image PIL的图像image
        :return: 旋转转之后的图像
        """
        random_angle = np.random.randint(1, 360)
        return image.rotate(random_angle, mode)

    @staticmethod
    def randomCrop(image):
        """
        对图像随意剪切,考虑到图像大小范围(68,68),使用一个一个大于(36*36)的窗口进行截图
        :param image: PIL的图像image
        :return: 剪切之后的图像

        """
        image_width = image.size[0]
        image_height = image.size[1]
        crop_win_size = np.random.randint(40, 68)
        random_region = (
            (image_width - crop_win_size) >> 1, (image_height - crop_win_size) >> 1, (image_width + crop_win_size) >> 1,
            (image_height + crop_win_size) >> 1)
        return image.crop(random_region)

    @staticmethod
    def randomColor(image):
        """
        对图像进行颜色抖动
        :param image: PIL的图像image
        :return: 有颜色色差的图像image
        """
        random_factor = np.random.randint(0, 31) / 10.  # 随机因子
        color_image = ImageEnhance.Color(image).enhance(random_factor)  # 调整图像的饱和度
        random_factor = np.random.randint(10, 21) / 10.  # 随机因子
        brightness_image = ImageEnhance.Brightness(color_image).enhance(random_factor)  # 调整图像的亮度
        random_factor = np.random.randint(10, 21) / 10.  # 随机因1子
        contrast_image = ImageEnhance.Contrast(brightness_image).enhance(random_factor)  # 调整图像对比度
        random_factor = np.random.randint(0, 31) / 10.  # 随机因子
        return ImageEnhance.Sharpness(contrast_image).enhance(random_factor)  # 调整图像锐度

    @staticmethod
    def randomGaussian(image, mean=0.2, sigma=0.3):
        """
         对图像进行高斯噪声处理
        :param image:
        :return:
        """

        def gaussianNoisy(im, mean=0.2, sigma=0.3):
            """
            对图像做高斯噪音处理
            :param im: 单通道图像
            :param mean: 偏移量
            :param sigma: 标准差
            :return:
            """
            for _i in range(len(im)):
                im[_i] += random.gauss(mean, sigma)
            return im

        # 将图像转化成数组
        img = np.asarray(image)
        img.flags.writeable = True  # 将数组改为读写模式
        width, height = img.shape[:2]
        img_r = gaussianNoisy(img[:, :, 0].flatten(), mean, sigma)
        img_g = gaussianNoisy(img[:, :, 1].flatten(), mean, sigma)
        img_b = gaussianNoisy(img[:, :, 2].flatten(), mean, sigma)
        img[:, :, 0] = img_r.reshape([width, height])
        img[:, :, 1] = img_g.reshape([width, height])
        img[:, :, 2] = img_b.reshape([width, height])
        return Image.fromarray(np.uint8(img))

    @staticmethod
    def saveImage(image, path):
        image.save(path)


def makeDir(path):
    try:
        if not os.path.exists(path):
            if not os.path.isfile(path):
                # os.mkdir(path)
                os.makedirs(path)
            return 0
        else:
            return 1
    except Exception, e:
        print str(e)
        return -2


def imageOps(func_name, image, des_path, file_name, times=5):
    funcMap = {"randomRotation": DataAugmentation.randomRotation,
               "randomCrop": DataAugmentation.randomCrop,
               "randomColor": DataAugmentation.randomColor,
               "randomGaussian": DataAugmentation.randomGaussian
               }
    if funcMap.get(func_name) is None:
        logger.error("%s is not exist", func_name)
        return -1

    for _i in range(0, times, 1):
        new_image = funcMap[func_name](image)
        DataAugmentation.saveImage(new_image, os.path.join(des_path, func_name + str(_i) + file_name))


opsList = {"randomRotation", "randomCrop", "randomColor", "randomGaussian"}


def threadOPS(path, new_path):
    """
    多线程处理事务
    :param src_path: 资源文件
    :param des_path: 目的地文件
    :return:
    """
    if os.path.isdir(path):
        img_names = os.listdir(path)
    else:
        img_names = [path]
    for img_name in img_names:
        print img_name
        tmp_img_name = os.path.join(path, img_name)
        if os.path.isdir(tmp_img_name):
            if makeDir(os.path.join(new_path, img_name)) != -1:
                threadOPS(tmp_img_name, os.path.join(new_path, img_name))
            else:
                print 'create new dir failure'
                return -1
                # os.removedirs(tmp_img_name)
        elif tmp_img_name.split('.')[1] != "DS_Store":
            # 读取文件并进行操作
            image = DataAugmentation.openImage(tmp_img_name)
            threadImage = [0] * 5
            _index = 0
            for ops_name in opsList:
                threadImage[_index] = threading.Thread(target=imageOps,
                                                       args=(ops_name, image, new_path, img_name,))
                threadImage[_index].start()
                _index += 1
                time.sleep(0.2)


if __name__ == '__main__':
    threadOPS("/home/pic-image/train/12306train",
              "/home/pic-image/train/12306train3")

SyntaxError: invalid syntax (<ipython-input-1-faa1a1823ef7>, line 125)

# **Traffic Sign Recognition** 

---

**Build a Traffic Sign Recognition Project**

The goals / steps of this project are the following:
* Load the data set (see below for links to the project data set)
* Explore, summarize and visualize the data set
* Design, train and test a model architecture
* Use the model to make predictions on new images
* Analyze the softmax probabilities of the new images
* Summarize the results with a written report


[//]: # (Image References)

[image11]: ./examples/all_class_traffic_types.png "all_class_traffic_types"
[image12]: ./examples/classes_distribution.jpg "classes_distribution"
[image2]: ./examples/src_gray.jpg "Grayscaling"
[image3]: ./examples/random_noise.jpg "Random Noise"
[image4]: ./examples/train_accuracy.jpg "train_accuracy"
[image5]: ./examples/New_Images.jpg "New_Images"
[image6]: ./examples/predict_images.jpg "predict_images"
[image7]: ./examples/Top_proba_new_images.jpg "Top_proba_new_images"
[image8]: ./examples/placeholder.png "Traffic Sign 5"

---


### Data Set Summary & Exploration

#### 1. Provide a basic summary of the data set. In the code, the analysis should be done using python, numpy and/or pandas methods rather than hardcoding results manually.

 * The size of training set, validtion set, and test set is:

    Training Set:   34799 samples

    Valid Set:     4410 samples

    Test Set:      12630 samples

 * The shape of a traffic sign image is 

    Image Shape: (32, 32, 3)

 * The number of unique classes/labels in the data set is 

    43

#### 2. Include an exploratory visualization of the dataset.

Here is an exploratory visualization of the data set. It is a bar chart showing how the data ...

![alt text][image11]

class contribution:

![alt text][image12]


## Design and Test a Model Architecture

### step1: Pre-process the Data Set

As a first step, I decided to convert the images to grayscale because the gray image works well in classification, and reduce the amount of calculation. 

Here is an example of a traffic sign image before and after grayscaling.

![alt text][image2]

As a last step, I normalized the image data because normalized data is easier to converge in training.



In addition, i decided to generate additional data because the data is unbalanced.(It's not work well in current project, will be update in feature)

To add more data to the the data set, I used the following techniques 

•	Slight random rotations

•	Adding random shadows

Here is an example of an original image and an augmented image:

#![image][image3]

The difference between the original data set and the augmented data set is the following ... 

### step2:  Design model architecture

my final model architecture looks like below, consisted of the following layers:

| Layer         		|     Description	        					| 
|:---------------------:|:---------------------------------------------:| 
| Input         		| 32x32x1 RGB image   							| 
| Convolution 3x3     	| 1x1 stride, valid padding, outputs 30x30x8 	|
| RELU					|												|
| Max pooling	      	| 2x2 stride,  outputs 15x15x8 				|
| Dropout           | 0.5|
| Convolution 3x3	    | 1x1 stride, valid padding, outputs 13x13x8   |
| RELU             |                               |
| Max pooling        | 2x2 stride, outputs  6x6x26
| Fully connected		| inputs 936, outputs 400        									|
| RELU             |                               |
| Fully connected     | inputs 400, outputs 120                           |
| RELU             |                               |
| Fully connected     | intputs 120, outputs 84                      |
| RELU             |                               |
| Fully connected     | inputs 84, outputs 43                      |
| Softmax				| 43x1        									|

 


### step3: Train model. 

To train the model, I used an AdamOptimizer, and  hyperparameters shows below

EPOCHS = 30

BATCH_SIZE = 128

rate = 0.001

when epoch is bigger than 10, set learn rate to 0.0001.


My final model results were:
* training set accuracy of 0.992
* validation set accuracy of 0.934
* test set accuracy of 0.935

validation set accuracy shows below:

![alt text][image4]

to train model, i take some steps:

First

### step4: Test a Model on New Images

#### 1. download ten German traffic signs  on the web;

Here are five German traffic signs that I found on the web:

![alt text][image5]

The first image might be difficult to classify because the sample of "Speed limit(30km/h)" in X_train is too little.

#### 2. Here are the results of the prediction:

![alt text][image6]

Here are the results of the prediction:

| Image			        |     Prediction	        					|  result|
|:---------------------:|:---------------------------------------------:|:---------------:|
| Speed limit (30km/h)  | Speed limit (50km/h)   					| false |
| Pedestrians     	    | Pedestrians 								| true  |
| Turn right ahead      | Speed limit (30km/h)   					| false |
| Go straight or left   | Go straight or left 						| true  |
| Speed limit (60km/h)  | Speed limit (60km/h)   					| true  |
| Children crossing     | Children crossing 						| true  |
| Stop                  | Speed limit (30km/h)   				    | false |
| Yield     			| Yield 									| true  | 
| Turn right ahead      | Turn right ahead   						| true  |
| Wild animals crossing | Wild animals crossing 					| true  |


The model was able to correctly guess 7 of the 10 traffic signs, which gives an accuracy of 70%. 

This is  worse than the accuracy of the test set.

#### 3.  the softmax probabilities for each prediction. 

For the first image, the model is relatively sure that this is a Speed limit (50km/h) sign (probability of 0.99), but the image does contain a Speed limit (30km/h) sign. it's wrong.

The top five soft max probabilities were

![alt text][image7]

## problem: i think the project has problems is:

1.train set is too small and unbalance.



### (Optional) Visualizing the Neural Network (See Step 4 of the Ipython notebook for more details)
#### 1. Discuss the visual output of your trained network's feature maps. What characteristics did the neural network use to make classifications?

